In [1]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

CPU times: user 8.19 s, sys: 6.04 s, total: 14.2 s
Wall time: 8.14 s


In [2]:
cloud_dataset = np.load('datasets/train_dataset/1M_regressor_train.npz')

In [3]:
X_train_set = cloud_dataset['X_train']
y_train_set = cloud_dataset['y_train']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_train_set, y_train_set, test_size=0.2, random_state=42)

In [5]:
def find_best_regression(x_train, y_train, model, param_grid, cv=5, scoring='neg_mean_squared_error'):
    # Initialize GridSearchCV with the model, parameter grid, and cross-validation settings
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=True)
    # Fit the grid search to the training data
    grid_search.fit(x_train, y_train)
    # Get the best model, parameters, score, and full CV results
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    cv_results = grid_search.cv_results_  # Includes performance metrics for each parameter combination
    
    return best_model, best_params, best_score, cv_results


In [14]:
from sklearn.linear_model import Ridge

regressors = {
        'random_forest': RandomForestRegressor(random_state=42),
        'svr': SVR(),
        'gradient_boosting': GradientBoostingRegressor(random_state=42),
        'linear_regression': Ridge(),
        'knn': KNeighborsRegressor(),
    }

param_grids_regressor = {
    'random_forest': {
        'n_estimators': [100, 300, 500, 1000],
        'max_depth': [6, 10, 15],
    },
    'svr': {
        'kernel': ['poly', 'linear'],
        'C': [0.1, 1, 10],
        'degree': [2, 3],  # For poly kernel
    },
    'gradient_boosting': {
        'n_estimators': [100, 300],
        'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 4, 5],
    },
    'linear_regression': {
        'alpha': [0.01, 0.1, 1, 10, 100],  # Regularization strength
        'fit_intercept': [True, False],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg'],
    },
    'knn': {
        'n_neighbors': [3, 5, 7, 10, 20, 30, 50],
        'weights': ['uniform', 'distance'],
        'p': [1, 2, 3],
    },
}


In [ ]:
%%time
best_model, best_params, best_score, cv_results = find_best_regression(X_train, 
                                                                       y_train, regressors['knn'], 
                                                                       param_grids_regressor['knn'])

In [ ]:
# Convert negative MSE back to positive by negating the test scores
# mse_test_scores = -cv_results['split0_test_score'], -cv_results['split1_test_score'], -cv_results['split2_test_score'], -cv_results['split3_test_score'], -cv_results['split4_test_score']

# Print best parameters, best result, and MSE over splits
print(f"Best parameters: {best_params}")
print(f"Best result (negative MSE): {-best_score}")
# print(f"MSE over splits (test): {mse_test_scores}")

In [ ]:
folder_path = 'model_train/new_regressor/'
file_path = os.path.join(folder_path, 'fill_nan_knn.pkl')
joblib.dump(best_model, file_path)

In [ ]:
y_pred = best_model.predict(X_test)

# Calculate R² score
r2 = r2_score(y_test, y_pred)
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"R² Score: {r2}")
print(f"Mean Squared Error: {mse}")

In [12]:
best_model = joblib.load('model_train/new_regressor/fill_nan_ln.pkl')